# 🎯 MAP Competition: Gemma-2-2b-it Submission Notebook

## Overview
このノートブックは、事前に訓練・保存されたGemma-2-2b-itモデルを使用して、MAP - Charting Student Math Misunderstandingsコンペティションの提出を行います。

### Model Details
- **Pre-trained Model**: google/gemma-2-2b-it
- **Parameters**: ~2.6B
- **Task**: 6-class text classification
- **Evaluation Metric**: MAP@3

### Target Classes (6分類)
- **True_Correct**: 正解で正しい説明
- **True_Neither**: 正解だが曖昧な説明
- **True_Misconception**: 正解だが誤った概念の説明
- **False_Correct**: 不正解だが正しい概念の説明
- **False_Neither**: 不正解で曖昧な説明
- **False_Misconception**: 不正解で誤った概念の説明

### Strategy
1. Kaggleにアップロードされた事前訓練済みモデルを読み込み
2. テストデータを前処理
3. 推論実行でMAP@3形式の予測生成
4. submission.csv作成・提出

## 📦 Install Required Libraries

In [1]:
# Kaggle環境で必要なライブラリをインストール
import subprocess
import sys

def install_package(package):
    """パッケージをインストールする関数"""
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# 必要なライブラリのインストール（Kaggleで通常不足するもの）
required_packages = [
    "transformers>=4.35.0",
    "accelerate>=0.26.0", 
    "sentencepiece>=0.1.99"
]

for package in required_packages:
    try:
        # インポートテストでインストール状況確認
        if "transformers" in package:
            import transformers
            print(f"✅ transformers already installed: {transformers.__version__}")
        elif "accelerate" in package:
            import accelerate
            print(f"✅ accelerate already installed: {accelerate.__version__}")
        elif "sentencepiece" in package:
            import sentencepiece
            print(f"✅ sentencepiece already installed: {sentencepiece.__version__}")
    except ImportError:
        print(f"📦 Installing {package}...")
        install_package(package)

print("🎉 All required libraries are ready!")

✅ transformers already installed: 4.52.4
✅ accelerate already installed: 1.8.1
✅ sentencepiece already installed: 0.2.0
🎉 All required libraries are ready!


## 📚 Import Dependencies

In [2]:
# 基本ライブラリ
import pandas as pd
import numpy as np
import os
import json
import time
import warnings
from pathlib import Path

# 機械学習ライブラリ
import torch
from torch.utils.data import Dataset, DataLoader

# Transformersライブラリ
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
)

# 警告を非表示
warnings.filterwarnings("ignore")

# デバッグ情報表示
print("🔧 Environment Information:")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU device: {torch.cuda.get_device_name(0)}")
    print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    device = torch.device("cuda")
else:
    print("Running on CPU")
    device = torch.device("cpu")

# Kaggleのデータパス設定
KAGGLE_INPUT_PATH = "/kaggle/input"
if os.path.exists(KAGGLE_INPUT_PATH):
    print(f"📁 Kaggle environment detected: {KAGGLE_INPUT_PATH}")
    # コンペティションデータパス
    COMP_DATA_PATH = "/kaggle/input/map-charting-student-math-misunderstandings"
    # アップロードしたモデルのパス（実際のデータセット名に変更してください）
    MODEL_DATA_PATH = "/kaggle/input/test_gemma-2-2b-math-misconception/transformers/default/1/gemma-2-2b-math-misconception"
    print(f"🎯 Competition data path: {COMP_DATA_PATH}")
    print(f"🤖 Model data path: {MODEL_DATA_PATH}")
else:
    print("📁 Local environment detected")
    COMP_DATA_PATH = "./map_data"
    MODEL_DATA_PATH = "./saved_models/gemma-2-2b-math-misconception"

print("✅ All libraries imported successfully!")

2025-07-21 09:42:37.625936: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753090957.990726      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753090958.093713      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🔧 Environment Information:
PyTorch version: 2.6.0+cu124
CUDA available: True
CUDA version: 12.4
GPU device: Tesla T4
GPU memory: 15.8 GB
📁 Kaggle environment detected: /kaggle/input
🎯 Competition data path: /kaggle/input/map-charting-student-math-misunderstandings
🤖 Model data path: /kaggle/input/test_gemma-2-2b-math-misconception/transformers/default/1/gemma-2-2b-math-misconception
✅ All libraries imported successfully!


## 🔧 Define Dataset Class

In [3]:
class MathMisconceptionDataset(Dataset):
    """
    Math Misconception Dataset for PyTorch
    推論専用のデータセット
    """
    
    def __init__(self, texts, tokenizer, max_length=512):
        """
        Args:
            texts (list): テキストデータのリスト
            tokenizer: Gemmaトークナイザー
            max_length (int): 最大トークン長
        """
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])

        # Gemmaトークナイザーでテキストをエンコード
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding="max_length",
            max_length=self.max_length,
            return_tensors="pt",
        )

        return {
            "input_ids": encoding["input_ids"].flatten(),
            "attention_mask": encoding["attention_mask"].flatten(),
        }

print("✅ MathMisconceptionDataset class defined successfully!")

✅ MathMisconceptionDataset class defined successfully!


## 🤖 Load Pre-trained Gemma Model

In [4]:
def load_pretrained_gemma_model():
    """事前訓練済みGemmaモデルの読み込み"""
    print("=" * 60)
    print("🤖 事前訓練済みGemmaモデル読み込み")
    print("=" * 60)
    
    try:
        # モデルパス確認
        print(f"📁 モデルパス: {MODEL_DATA_PATH}")
        
        if not os.path.exists(MODEL_DATA_PATH):
            print(f"❌ モデルパスが存在しません: {MODEL_DATA_PATH}")
            print("💡 Kaggle環境では正しいデータセット名を確認してください")
            return None, None, None
        
        # ラベルマッピングの読み込み
        label_file = os.path.join(MODEL_DATA_PATH, "label_mapping.json")
        print(f"📋 ラベルマッピング読み込み: {label_file}")
        
        with open(label_file, "r", encoding="utf-8") as f:
            label_mapping = json.load(f)
        
        print("✅ ラベルマッピング読み込み成功")
        for idx, label in label_mapping.items():
            print(f"   {idx}: {label}")
        
        # トークナイザーの読み込み
        print("\n📝 Gemmaトークナイザー読み込み中...")
        tokenizer = AutoTokenizer.from_pretrained(MODEL_DATA_PATH)
        print(f"✅ トークナイザー読み込み成功")
        print(f"🔖 パディングトークン: {tokenizer.pad_token}")
        print(f"📏 語彙サイズ: {tokenizer.vocab_size:,}")
        
        # モデルの読み込み
        print("\n🧠 Gemmaモデル読み込み中...")
        model = AutoModelForSequenceClassification.from_pretrained(
            MODEL_DATA_PATH,
            torch_dtype=torch.float16 if device.type == "cuda" else torch.float32,
            device_map="auto" if device.type == "cuda" else None,
        )
        
        # デバイスに移動（必要に応じて）
        if device.type == "cpu":
            model = model.to(device)
        
        print(f"✅ Gemmaモデル読み込み完了!")
        
        # モデル情報表示
        total_params = sum(p.numel() for p in model.parameters())
        print(f"📊 分類クラス数: {model.config.num_labels}")
        print(f"📈 総パラメータ数: {total_params:,}")
        print(f"💡 モデルサイズ: ~{total_params / 1e9:.2f}B parameters")
        
        # 推論モードに設定
        model.eval()
        
        return model, tokenizer, label_mapping
    
    except Exception as e:
        print(f"❌ モデル読み込みエラー: {e}")
        print("\n🔧 トラブルシューティング:")
        print("1. Kaggleでモデルデータセットが正しくアップロードされているか確認")
        print("2. MODEL_DATA_PATHが正しいデータセット名を指しているか確認")
        print("3. 必要なファイルが全て含まれているか確認")
        raise e

# モデル読み込み実行
model, tokenizer, label_mapping = load_pretrained_gemma_model()
print("\n🎉 事前訓練済みモデル準備完了!")

🤖 事前訓練済みGemmaモデル読み込み
📁 モデルパス: /kaggle/input/test_gemma-2-2b-math-misconception/transformers/default/1/gemma-2-2b-math-misconception
📋 ラベルマッピング読み込み: /kaggle/input/test_gemma-2-2b-math-misconception/transformers/default/1/gemma-2-2b-math-misconception/label_mapping.json
✅ ラベルマッピング読み込み成功
   0: False_Correct
   1: False_Misconception
   2: False_Neither
   3: True_Correct
   4: True_Misconception
   5: True_Neither

📝 Gemmaトークナイザー読み込み中...
✅ トークナイザー読み込み成功
🔖 パディングトークン: <pad>
📏 語彙サイズ: 256,000

🧠 Gemmaモデル読み込み中...


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

✅ Gemmaモデル読み込み完了!
📊 分類クラス数: 6
📈 総パラメータ数: 2,614,355,712
💡 モデルサイズ: ~2.61B parameters

🎉 事前訓練済みモデル準備完了!


## 📊 Load and Prepare Test Data

In [5]:
def load_and_prepare_test_data():
    """テストデータの読み込みと前処理"""
    print("=" * 60)
    print("📊 テストデータ読み込みと前処理")
    print("=" * 60)
    
    try:
        # テストデータの読み込み
        test_path = os.path.join(COMP_DATA_PATH, "test.csv")
        print(f"📁 テストデータパス: {test_path}")
        
        if not os.path.exists(test_path):
            print(f"❌ テストデータが見つかりません: {test_path}")
            return None
        
        test_df = pd.read_csv(test_path)
        print(f"✅ テストデータ読み込み成功!")
        print(f"📈 テストデータ形状: {test_df.shape}")
        
        # データ確認
        print("\n📋 テストデータの列:")
        print(test_df.columns.tolist())
        
        print("\n📋 データサンプル:")
        print(test_df.head(3))
        
        # 強化されたテキスト特徴量の作成
        def create_enhanced_text(row):
            """Question + MC_Answer + Explanation を結合した強化テキスト"""
            question = str(row.get("QuestionText", "")) if pd.notna(row.get("QuestionText")) else ""
            mc_answer = str(row.get("MC_Answer", "")) if pd.notna(row.get("MC_Answer")) else ""
            explanation = str(row.get("StudentExplanation", "")) if pd.notna(row.get("StudentExplanation")) else ""
            
            # Gemma用の構造化テキスト
            enhanced_text = f"Question: {question} Selected Answer: {mc_answer} Explanation: {explanation}"
            return enhanced_text

        print("\n🔧 強化テキスト特徴量作成中...")
        test_df["enhanced_text"] = test_df.apply(create_enhanced_text, axis=1)
        
        # テキスト長の統計
        text_lengths = test_df["enhanced_text"].str.len()
        print(f"\n📊 テキスト長統計:")
        print(f"   平均: {text_lengths.mean():.0f} 文字")
        print(f"   最小: {text_lengths.min()} 文字")
        print(f"   最大: {text_lengths.max()} 文字")
        print(f"   中央値: {text_lengths.median():.0f} 文字")
        
        # サンプルテキストの表示
        print(f"\n📝 強化テキストサンプル:")
        sample_text = test_df["enhanced_text"].iloc[0]
        print(f"Length: {len(sample_text)} characters")
        print(f"Sample: {sample_text[:300]}...")
        
        return test_df
    
    except Exception as e:
        print(f"❌ テストデータ読み込みエラー: {e}")
        return None

# テストデータ読み込み実行
test_df = load_and_prepare_test_data()

📊 テストデータ読み込みと前処理
📁 テストデータパス: /kaggle/input/map-charting-student-math-misunderstandings/test.csv
✅ テストデータ読み込み成功!
📈 テストデータ形状: (3, 5)

📋 テストデータの列:
['row_id', 'QuestionId', 'QuestionText', 'MC_Answer', 'StudentExplanation']

📋 データサンプル:
   row_id  QuestionId                                       QuestionText  \
0   36696       31772  What fraction of the shape is not shaded? Give...   
1   36697       31772  What fraction of the shape is not shaded? Give...   
2   36698       32835                      Which number is the greatest?   

           MC_Answer                                 StudentExplanation  
0  \( \frac{1}{3} \)  I think that 1/3 is the answer, as it's the si...  
1  \( \frac{3}{6} \)  i think this answer is because 3 triangles are...  
2          \( 6.2 \)     because the 2 makes it higher than the others.  

🔧 強化テキスト特徴量作成中...

📊 テキスト長統計:
   平均: 235 文字
   最小: 126 文字
   最大: 296 文字
   中央値: 284 文字

📝 強化テキストサンプル:
Length: 284 characters
Sample: Question: What fraction of the sh

## 🔮 Generate Predictions for Test Set

In [6]:
def generate_test_predictions(model, tokenizer, test_df, label_mapping, batch_size=8):
    """テストセットに対する予測生成（MAP@3形式）"""
    print("=" * 60)
    print("🔮 テストセット予測生成（MAP@3）")
    print("=" * 60)
    
    if test_df is None or len(test_df) == 0:
        print("❌ テストデータが利用できません")
        return None
    
    print(f"📊 テストデータ: {len(test_df):,}件")
    print(f"🔧 バッチサイズ: {batch_size}")
    
    # テストデータの前処理
    test_texts = test_df["enhanced_text"].tolist()
    
    # テストデータセット作成
    print("🔧 テストデータセット作成中...")
    test_dataset = MathMisconceptionDataset(
        test_texts, tokenizer, max_length=512
    )
    
    # データローダー作成
    test_dataloader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False,
        collate_fn=DataCollatorWithPadding(tokenizer=tokenizer)
    )
    
    print(f"✅ テストデータローダー作成: {len(test_dataloader)}バッチ")
    
    # 予測実行
    print("🔮 予測実行中...")
    all_predictions = []
    
    try:
        model.eval()
        with torch.no_grad():
            for batch_idx, batch in enumerate(test_dataloader):
                # バッチをデバイスに移動
                batch = {k: v.to(device) for k, v in batch.items()}
                
                # 推論実行
                outputs = model(**batch)
                predictions = outputs.logits
                
                # CPU に移動してリストに追加
                batch_predictions = predictions.cpu().numpy()
                all_predictions.append(batch_predictions)
                
                # 進捗表示
                if (batch_idx + 1) % 10 == 0 or (batch_idx + 1) == len(test_dataloader):
                    processed = min((batch_idx + 1) * batch_size, len(test_df))
                    print(f"   進捗: {processed:,}/{len(test_df):,} ({processed/len(test_df)*100:.1f}%)")
        
        print("✅ 予測完了!")
        
        # 予測結果を結合
        all_predictions = np.vstack(all_predictions)
        print(f"📊 予測結果形状: {all_predictions.shape}")
        
        # 確率に変換
        probs = torch.softmax(torch.tensor(all_predictions), dim=-1).numpy()
        
        # 各サンプルで上位3つの予測を取得
        print("🎯 TOP-3予測抽出中...")
        submission_predictions = []
        
        # ラベルマッピングの逆変換用
        idx_to_label = {int(k): v for k, v in label_mapping.items()}
        
        for i, prob in enumerate(probs):
            # 確率の高い順に上位3つのインデックスを取得
            top3_indices = np.argsort(prob)[::-1][:3]
            
            # インデックスを実際のラベル名に変換
            top3_labels = [idx_to_label[idx] for idx in top3_indices]
            
            # スペース区切りで結合（コンペ要求形式）
            prediction_string = " ".join(top3_labels)
            submission_predictions.append(prediction_string)
            
            # 進捗表示（最初の5件）
            if i < 5:
                top3_probs = [prob[idx] for idx in top3_indices]
                print(f"  サンプル {i+1}: {prediction_string}")
                print(f"    確率: {[f'{p:.3f}' for p in top3_probs]}")
        
        print(f"✅ TOP-3予測抽出完了: {len(submission_predictions)}件")
        
        # 予測の統計情報
        all_pred_labels = " ".join(submission_predictions).split()
        from collections import Counter
        pred_counts = Counter(all_pred_labels)
        
        print(f"\n📈 予測統計:")
        print(f"  予測に使用されたカテゴリ数: {len(pred_counts)}")
        for category, count in pred_counts.most_common():
            percentage = count / (len(submission_predictions) * 3) * 100
            print(f"    {category}: {count}回 ({percentage:.1f}%)")
        
        return submission_predictions
    
    except Exception as e:
        print(f"❌ 予測実行中にエラー: {e}")
        if torch.cuda.is_available():
            print(f"🖥️ 現在のGPUメモリ使用量: {torch.cuda.memory_allocated() / 1e6:.1f} MB")
        raise e

# 予測実行
if model is not None and tokenizer is not None and test_df is not None:
    print("🔮 テストセット予測を開始します...")
    # バッチサイズを環境に応じて調整
    batch_size = 4 if device.type == "cpu" else 8
    test_predictions = generate_test_predictions(model, tokenizer, test_df, label_mapping, batch_size)
    print("🎉 テスト予測完了!")
else:
    print("❌ 必要なコンポーネントが準備されていません")
    test_predictions = None

🔮 テストセット予測を開始します...
🔮 テストセット予測生成（MAP@3）
📊 テストデータ: 3件
🔧 バッチサイズ: 8
🔧 テストデータセット作成中...
✅ テストデータローダー作成: 1バッチ
🔮 予測実行中...
   進捗: 3/3 (100.0%)
✅ 予測完了!
📊 予測結果形状: (3, 6)
🎯 TOP-3予測抽出中...
  サンプル 1: True_Neither True_Misconception False_Correct
    確率: ['0.965', '0.027', '0.005']
  サンプル 2: True_Neither True_Misconception False_Correct
    確率: ['0.937', '0.041', '0.018']
  サンプル 3: True_Neither True_Misconception False_Correct
    確率: ['0.537', '0.435', '0.016']
✅ TOP-3予測抽出完了: 3件

📈 予測統計:
  予測に使用されたカテゴリ数: 3
    True_Neither: 3回 (33.3%)
    True_Misconception: 3回 (33.3%)
    False_Correct: 3回 (33.3%)
🎉 テスト予測完了!


## 📤 Create Submission File

最終ステップとして、Kaggleコンペティションに提出するためのCSVファイルを作成します。

In [7]:
def create_submission_file(test_df, predictions, output_path="submission.csv"):
    """提出用CSVファイルの作成"""
    print("=" * 60)
    print("📤 提出ファイル作成")
    print("=" * 60)
    
    if test_df is None or predictions is None:
        print("❌ テストデータまたは予測結果がありません")
        return None
    
    print(f"📊 提出データ: {len(predictions):,}件")
    
    # sample_submission.csvを参考にして正しい形式で作成
    # row_id,Category:Misconception の形式が必要
    
    # テストデータにrow_idが含まれているか確認
    if 'row_id' in test_df.columns:
        row_ids = test_df['row_id'].tolist()
    else:
        # row_idがない場合、testデータの開始IDを推定（通常は36696から）
        print("⚠️ row_idが見つかりません。推定値を使用します...")
        start_id = 36696  # sample_submissionの開始ID
        row_ids = list(range(start_id, start_id + len(test_df)))
    
    # Kaggle要求形式：各予測を "Category:Misconception" 形式に変換
    print("🔧 Kaggle提出形式に変換中...")
    formatted_predictions = []
    
    for i, pred in enumerate(predictions):
        # predは "True_Correct False_Neither False_Misconception" のようなスペース区切り
        pred_categories = pred.split()[:3]  # TOP3に限定
        
        # 各カテゴリを "Category:Misconception" 形式に変換
        formatted_parts = []
        for category in pred_categories:
            if category.endswith('_Misconception'):
                # Misconceptionカテゴリの場合、実際の誤概念名が必要
                # ここでは簡易的に "Incomplete" を使用（実際は予測結果から取得）
                formatted_parts.append(f"{category}:Incomplete")
            else:
                # 他のカテゴリの場合はNA
                formatted_parts.append(f"{category}:NA")
        
        # スペース区切りで結合
        formatted_pred = " ".join(formatted_parts)
        formatted_predictions.append(formatted_pred)
        
        # 最初の5件をサンプル表示
        if i < 5:
            print(f"  サンプル {i+1}: {formatted_pred}")
    
    # 提出用データフレーム作成（正しいKaggle形式）
    submission_df = pd.DataFrame({
        'row_id': row_ids,
        'Category:Misconception': formatted_predictions
    })
    
    print(f"✅ 提出データフレーム作成完了: {submission_df.shape}")
    print(f"📝 列: {list(submission_df.columns)}")
    
    # サンプル表示
    print(f"\n📋 提出データサンプル:")
    print(submission_df.head(10).to_string(index=False))
    
    # ファイル保存
    try:
        submission_df.to_csv(output_path, index=False)
        print(f"\n💾 提出ファイル保存完了: {output_path}")
        
        # ファイルサイズ確認
        file_size = os.path.getsize(output_path)
        print(f"📏 ファイルサイズ: {file_size:,} bytes ({file_size/1024:.1f} KB)")
        
        # 形式確認
        check_df = pd.read_csv(output_path)
        print(f"✅ 提出ファイル検証: {check_df.shape}")
        required_cols = ['row_id', 'Category:Misconception']
        cols_present = all(col in check_df.columns for col in required_cols)
        print(f"   必要列存在確認: {cols_present}")
        
        # 予測形式チェック
        sample_predictions = check_df['Category:Misconception'].head(5).tolist()
        print(f"🔍 予測形式サンプル:")
        for i, pred in enumerate(sample_predictions):
            pred_parts = pred.split()
            print(f"   {i+1}: {pred} (要素数: {len(pred_parts)})")
            
        print(f"\n📊 提出ファイル最終確認:")
        print(f"   ✅ 形式: Kaggle MAP競技形式（row_id, Category:Misconception）")
        print(f"   ✅ 件数: {len(check_df):,}件")
        print(f"   ✅ 列名: {list(check_df.columns)}")
        
        return submission_df
    
    except Exception as e:
        print(f"❌ ファイル保存エラー: {e}")
        return None

# 提出ファイルの作成
if test_predictions is not None and test_df is not None:
    print("📤 提出ファイルを作成します...")
    submission_df = create_submission_file(test_df, test_predictions, "submission.csv")
    
    if submission_df is not None:
        print("🎉 提出ファイル作成完了!")
        print(f"📋 最終確認:")
        print(f"   ファイル名: submission.csv")
        print(f"   データ件数: {len(submission_df):,}")
        print(f"   予測形式: Kaggle MAP形式 (row_id, Category:Misconception)")
        print("\n🏆 Kaggleコンペティションに提出準備完了!")
        
        # 最終チェック用統計
        print(f"\n📈 提出ファイル統計:")
        all_pred_in_submission = " ".join(submission_df['Category:Misconception']).split()
        unique_preds = set(all_pred_in_submission)
        print(f"   使用されたユニークなラベル数: {len(unique_preds)}")
        print(f"   ラベル一覧: {sorted(unique_preds)}")
    else:
        print("❌ 提出ファイル作成に失敗しました")
else:
    print("❌ 必要なデータが準備されていません。前のセルを実行してください。")

📤 提出ファイルを作成します...
📤 提出ファイル作成
📊 提出データ: 3件
🔧 Kaggle提出形式に変換中...
  サンプル 1: True_Neither:NA True_Misconception:Incomplete False_Correct:NA
  サンプル 2: True_Neither:NA True_Misconception:Incomplete False_Correct:NA
  サンプル 3: True_Neither:NA True_Misconception:Incomplete False_Correct:NA
✅ 提出データフレーム作成完了: (3, 2)
📝 列: ['row_id', 'Category:Misconception']

📋 提出データサンプル:
 row_id                                         Category:Misconception
  36696 True_Neither:NA True_Misconception:Incomplete False_Correct:NA
  36697 True_Neither:NA True_Misconception:Incomplete False_Correct:NA
  36698 True_Neither:NA True_Misconception:Incomplete False_Correct:NA

💾 提出ファイル保存完了: submission.csv
📏 ファイルサイズ: 237 bytes (0.2 KB)
✅ 提出ファイル検証: (3, 2)
   必要列存在確認: True
🔍 予測形式サンプル:
   1: True_Neither:NA True_Misconception:Incomplete False_Correct:NA (要素数: 3)
   2: True_Neither:NA True_Misconception:Incomplete False_Correct:NA (要素数: 3)
   3: True_Neither:NA True_Misconception:Incomplete False_Correct:NA (要素数: 3)

📊 提出ファイル最終確認:

## 🎯 Summary & Performance Notes

### 📊 実行結果サマリー
このノートブックでは以下を実装しました：

1. **🤖 事前訓練済みモデル読み込み**: KaggleにアップロードされたGemma-2-2b-itモデル
2. **📊 テストデータ処理**: math misconceptionコンペティションのtest.csv
3. **🔮 効率的な推論**: バッチ処理による高速予測
4. **📈 MAP@3形式出力**: TOP-3予測の生成
5. **📤 提出準備**: Kaggle形式のsubmission.csv作成

### 🚀 使用技術
- **モデル**: `google/gemma-2-2b-it` (~2.6B parameters)
- **フレームワーク**: PyTorch + Transformers
- **推論**: 事前訓練済みモデル使用（追加訓練なし）
- **評価**: MAP@3 (Mean Average Precision at 3)

### ⚡ パフォーマンス最適化
- バッチ処理による効率的な推論
- 適応的バッチサイズ（CPU: 4, GPU: 8）
- FP16使用（GPU環境）
- メモリ効率的なデータローダー

### 📝 使用方法
1. Kaggleで事前訓練済みモデルをデータセットとしてアップロード
2. MODEL_DATA_PATHを正しいデータセット名に変更
3. 全セルを順番に実行
4. `Gemma_2b_submission.csv`が生成される
5. KaggleコンペティションにCSVファイルを提出

### 🔧 トラブルシューティング
- **メモリ不足**: バッチサイズを下げる
- **モデル読み込み失敗**: データセット名とパスを確認
- **CUDA OOM**: CPUモードに切り替え

**🏆 Good luck with your submission!**